## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [1]:
#@title Check GPU
!nvidia-smi

Fri Jun  9 01:39:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   48C    P8    18W / 125W |      6MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
# !python -m pip install -U pip wheel
# %pip install -U ipython 

# #@markdown Branch (for development)
# BRANCH = "none" #@param {"type": "string"}
# if BRANCH == "none":
#     %pip install -U so-vits-svc-fork
# else:
#     %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

## Preprocessing

In [2]:
import os, shutil
from os.path import dirname, abspath
from utils.demucs.utils import separate

import librosa  # Optional. Use any library you like to read audio files.
import soundfile  # Optional. Use any library you like to write audio files.
from utils.audioslicer.slicer import Slicer
from utils.audiosplitter import audiosplitter

def preprocess(audiofile):
   # audiofile = '../dataset_raw/ssk/speaking/ssk_podcast_20230825.mp3'
   filename, ext = os.path.splitext(os.path.basename(audiofile))

   # create output dir
   output_dir = os.path.join('/home/arvin/so-vits-svc-fork/dataset_processed', filename)
   demucs_dir = os.path.join(output_dir, 'demucs')
   chunks_dir = os.path.join(output_dir, 'chunks_clean')

   # Create folders
   for path in [output_dir, demucs_dir, chunks_dir]:
      if not os.path.exists(path):
         os.makedirs(path, exist_ok=True)

   # Copy raw file to output folder root
   shutil.copy(audiofile, output_dir)

   # DEMUCS
   separate(inp=output_dir, outp=demucs_dir)

   # SILENCE SLICER
   demucs_model = 'htdemucs'
   audiofile = os.path.join(demucs_dir, demucs_model, filename, 'vocals.mp3')
   audio, sr = librosa.load(audiofile, sr=None, mono=False)  # Load an audio file with librosa.
   slicer = Slicer(
      sr=sr,
      db_threshold=-40,
      min_length=5000,
      win_l = 300, # min_interval=300,
      win_s = 10, # hop_size=10,
      max_silence_kept=200
   )
   chunks = slicer.slice(audio)
   for i, chunk in enumerate(chunks):
      if len(chunk.shape) > 1:
         chunk = chunk.T  # Swap axes if the audio is stereo.
      soundfile.write(f'{chunks_dir}/{filename}_{i}.wav', chunk, sr)  # Save sliced audio files with soundfile.

   # 10SEC SPLITTER
   audiosplitter.run_audiosplitter(input_directory=chunks_dir)

   return
   # split_audio_file(chunks_dir, segment_duration=10)      

In [3]:
# audiofile = '../dataset_raw/ssk/speaking/ssk_podcast_20230825.mp3'
# audiofile = '../dataset_raw/ssk/singing/성시경_넌_감동이었어.mp3'
# audiofile = '../dataset_raw/ssk/singing/성시경_너의_모든_순간.mp3'
# audiofile = '../dataset_raw/ssk/singing/성시경_제주도의_푸른_밤.mp3'

#SSK
# files = ['../dataset_raw/ssk/singing/성시경_두_사람.mp3',
#         '../dataset_raw/ssk/singing/성시경_사랑하는일.mp3',
#         '../dataset_raw/ssk/singing/성시경_세_사람.mp3',
#         '../dataset_raw/ssk/singing/성시경_안녕_나의_사랑.mp3',
#         '../dataset_raw/ssk/singing/성시경_희재.mp3']

# EDSHEERAN
# !audioconvert convert /home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/ /home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/ --output-format .mp3

# files = ['/home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/06 Photograph.mp3']

# audiofile = '/home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/06 Photograph.mp3'
# preprocess(audiofile)
import glob
files = glob.glob('/home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/*.mp3')
for audiofile in files:
    print('='*120)
    print('FILE:', audiofile)
    print('='*120)    
    preprocess(audiofile)


FILE: /home/arvin/so-vits-svc-fork/dataset_source/edsheeran_x/16 I See Fire.mp3
Going to separate the files:
/home/arvin/so-vits-svc-fork/dataset_processed/16 I See Fire/16 I See Fire.mp3
With command:  python3 -m demucs.separate -o /home/arvin/so-vits-svc-fork/dataset_processed/16 I See Fire/demucs -n htdemucs --mp3 --mp3-bitrate=320
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/arvin/so-vits-svc-fork/dataset_processed/16 I See Fire/demucs/htdemucs
Separating track /home/arvin/so-vits-svc-fork/dataset_processed/16 I See Fire/16 I See Fire.mp3


100%|████████████████████████████████████████████████████████████████████████| 304.2/304.2 [00:12<00:00, 25.33seconds/s]


executing 'slice' cost 18.144s
Segment 1/1 saved: 16 I See Fire_2_1.wav
Segment 1/2 saved: 16 I See Fire_8_1.wav
Segment 2/2 saved: 16 I See Fire_8_2.wav
Segment 1/1 saved: 16 I See Fire_6_1.wav
Segment 1/2 saved: 16 I See Fire_13_1.wav
Segment 2/2 saved: 16 I See Fire_13_2.wav
Segment 1/2 saved: 16 I See Fire_7_1.wav
Segment 2/2 saved: 16 I See Fire_7_2.wav
Segment 1/2 saved: 16 I See Fire_17_1.wav
Segment 2/2 saved: 16 I See Fire_17_2.wav
Segment 1/1 saved: 16 I See Fire_0_1.wav
Segment 1/2 saved: 16 I See Fire_11_1.wav
Segment 2/2 saved: 16 I See Fire_11_2.wav
Segment 1/2 saved: 16 I See Fire_16_1.wav
Segment 2/2 saved: 16 I See Fire_16_2.wav
Segment 1/2 saved: 16 I See Fire_4_1.wav
Segment 2/2 saved: 16 I See Fire_4_2.wav
Segment 1/1 saved: 16 I See Fire_9_1.wav
Segment 1/2 saved: 16 I See Fire_12_1.wav
Segment 2/2 saved: 16 I See Fire_12_2.wav
Segment 1/1 saved: 16 I See Fire_3_1.wav
Segment 1/1 saved: 16 I See Fire_1_1.wav
Segment 1/3 saved: 16 I See Fire_18_1.wav
Segment 2/3 sav

100%|████████████████████████████████████████████████████████████████████████| 315.9/315.9 [00:12<00:00, 25.45seconds/s]


executing 'slice' cost 19.042s
Segment 1/2 saved: 12 Afire Love_7_1.wav
Segment 2/2 saved: 12 Afire Love_7_2.wav
Segment 1/1 saved: 12 Afire Love_0_1.wav
Segment 1/3 saved: 12 Afire Love_10_1.wav
Segment 2/3 saved: 12 Afire Love_10_2.wav
Segment 3/3 saved: 12 Afire Love_10_3.wav
Segment 1/3 saved: 12 Afire Love_4_1.wav
Segment 2/3 saved: 12 Afire Love_4_2.wav
Segment 3/3 saved: 12 Afire Love_4_3.wav
Segment 1/1 saved: 12 Afire Love_1_1.wav
Segment 1/1 saved: 12 Afire Love_9_1.wav
Segment 1/2 saved: 12 Afire Love_2_1.wav
Segment 2/2 saved: 12 Afire Love_2_2.wav
Segment 1/1 saved: 12 Afire Love_6_1.wav
Segment 1/1 saved: 12 Afire Love_8_1.wav
Segment 1/2 saved: 12 Afire Love_11_1.wav
Segment 2/2 saved: 12 Afire Love_11_2.wav
Segment 1/9 saved: 12 Afire Love_12_1.wav
Segment 2/9 saved: 12 Afire Love_12_2.wav
Segment 3/9 saved: 12 Afire Love_12_3.wav
Segment 4/9 saved: 12 Afire Love_12_4.wav
Segment 5/9 saved: 12 Afire Love_12_5.wav
Segment 6/9 saved: 12 Afire Love_12_6.wav
Segment 7/9 sav

...

executing 'slice' cost 15.868s
Segment 1/2 saved: 06 Photograph_3_1.wav
Segment 2/2 saved: 06 Photograph_3_2.wav
Segment 1/2 saved: 06 Photograph_2_1.wav
Segment 2/2 saved: 06 Photograph_2_2.wav
Segment 1/1 saved: 06 Photograph_19_1.wav
Segment 1/1 saved: 06 Photograph_8_1.wav
Segment 1/5 saved: 06 Photograph_17_1.wav
Segment 2/5 saved: 06 Photograph_17_2.wav
Segment 3/5 saved: 06 Photograph_17_3.wav
Segment 4/5 saved: 06 Photograph_17_4.wav
Segment 5/5 saved: 06 Photograph_17_5.wav
Segment 1/2 saved: 06 Photograph_18_1.wav
Segment 2/2 saved: 06 Photograph_18_2.wav
Segment 1/1 saved: 06 Photograph_6_1.wav
Segment 1/2 saved: 06 Photograph_7_1.wav
Segment 2/2 saved: 06 Photograph_7_2.wav
Segment 1/1 saved: 06 Photograph_10_1.wav
Segment 1/1 saved: 06 Photograph_12_1.wav
Segment 1/1 saved: 06 Photograph_0_1.wav
Segment 1/2 saved: 06 Photograph_11_1.wav
Segment 2/2 saved: 06 Photograph_11_2.wav
Segment 1/1 saved: 06 Photograph_9_1.wav
Segment 1/1 saved: 06 Photograph_13_1.wav
Segment 1/1 s

100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 24.98seconds/s]


executing 'slice' cost 14.762s
Segment 1/1 saved: 08 Tenerife Sea_7_1.wav
Segment 1/1 saved: 08 Tenerife Sea_21_1.wav
Segment 1/1 saved: 08 Tenerife Sea_0_1.wav
Segment 1/1 saved: 08 Tenerife Sea_27_1.wav
Segment 1/1 saved: 08 Tenerife Sea_18_1.wav
Segment 1/1 saved: 08 Tenerife Sea_12_1.wav
Segment 1/1 saved: 08 Tenerife Sea_6_1.wav
Segment 1/1 saved: 08 Tenerife Sea_17_1.wav
Segment 1/1 saved: 08 Tenerife Sea_8_1.wav
Segment 1/1 saved: 08 Tenerife Sea_2_1.wav
Segment 1/1 saved: 08 Tenerife Sea_24_1.wav
Segment 1/1 saved: 08 Tenerife Sea_3_1.wav
Segment 1/1 saved: 08 Tenerife Sea_26_1.wav
Segment 1/1 saved: 08 Tenerife Sea_13_1.wav
Segment 1/1 saved: 08 Tenerife Sea_23_1.wav
Segment 1/1 saved: 08 Tenerife Sea_9_1.wav
Segment 1/1 saved: 08 Tenerife Sea_5_1.wav
Segment 1/1 saved: 08 Tenerife Sea_14_1.wav
Segment 1/1 saved: 08 Tenerife Sea_11_1.wav
Segment 1/1 saved: 08 Tenerife Sea_20_1.wav
Segment 1/1 saved: 08 Tenerife Sea_1_1.wav
Segment 1/1 saved: 08 Tenerife Sea_25_1.wav
Segment 1/

100%|██████████████████████████████████████████████| 193.04999999999998/193.04999999999998 [00:07<00:00, 24.49seconds/s]


executing 'slice' cost 11.503s
Segment 1/4 saved: 14 Shirtsleeves_8_1.wav
Segment 2/4 saved: 14 Shirtsleeves_8_2.wav
Segment 3/4 saved: 14 Shirtsleeves_8_3.wav
Segment 4/4 saved: 14 Shirtsleeves_8_4.wav
Segment 1/1 saved: 14 Shirtsleeves_9_1.wav
Segment 1/3 saved: 14 Shirtsleeves_4_1.wav
Segment 2/3 saved: 14 Shirtsleeves_4_2.wav
Segment 3/3 saved: 14 Shirtsleeves_4_3.wav
Segment 1/1 saved: 14 Shirtsleeves_5_1.wav
Segment 1/1 saved: 14 Shirtsleeves_0_1.wav
Segment 1/3 saved: 14 Shirtsleeves_10_1.wav
Segment 2/3 saved: 14 Shirtsleeves_10_2.wav
Segment 3/3 saved: 14 Shirtsleeves_10_3.wav
Segment 1/1 saved: 14 Shirtsleeves_6_1.wav
Segment 1/1 saved: 14 Shirtsleeves_2_1.wav
Segment 1/1 saved: 14 Shirtsleeves_1_1.wav
Segment 1/2 saved: 14 Shirtsleeves_7_1.wav
Segment 2/2 saved: 14 Shirtsleeves_7_2.wav
Segment 1/1 saved: 14 Shirtsleeves_3_1.wav


## Training

In [4]:
#@title Make dataset directory
!mkdir -p "../dataset_raw"

In [ ]:
# !rm -r "dataset_raw"
# !rm -r "dataset/44k"

In [9]:
# Copy files
train_files = glob.glob('/home/arvin/so-vits-svc-fork/dataset_processed/*/chunks_clean_10/*.wav')
for file in train_files:
    shutil.copy(file, '/home/arvin/so-vits-svc-fork/dataset_raw/edsheeran/')

In [ ]:
#@title Copy your dataset
#@markdown **We assume that your dataset is in your Google Drive's `so-vits-svc-fork/dataset/(speaker_name)` directory.**
# DATASET_NAME = "edsheeran" #@param {type: "string"}
# !cp -R /{DATASET_NAME}/' -t "dataset_raw/"

In [ ]:
#@title Download dataset (Tsukuyomi-chan JVS)
#@markdown You can download this dataset if you don't have your own dataset.
#@markdown Make sure you agree to the license when using this dataset.
#@markdown https://tyc.rei-yumesaki.net/material/corpus/#toc6
# !wget https://tyc.rei-yumesaki.net/files/sozai-tyc-corpus1.zip
# !unzip sozai-tyc-corpus1.zip
# !mv "/content/つくよみちゃんコーパス Vol.1 声優統計コーパス（JVSコーパス準拠）/おまけ：WAV（+12dB増幅＆高音域削減）/WAV（+12dB増幅＆高音域削減）" "dataset_raw/tsukuyomi"

In [ ]:
#@title Automatic preprocessing
!svc pre-resample

In [ ]:
!svc pre-config

In [ ]:
#@title Copy configs file
# !cp configs/44k/config.json drive/MyDrive/so-vits-svc-fork

In [ ]:
F0_METHOD = "crepe" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
!svc pre-hubert -fm {F0_METHOD}

In [ ]:
#@title Train
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/so-vits-svc-fork/logs/44k
!svc train --model-path drive/MyDrive/so-vits-svc-fork/logs/44k

## Training Cluster model

In [ ]:
!svc train-cluster --output-path drive/MyDrive/so-vits-svc-fork/logs/44k/kmeans.pt

## Inference

In [ ]:
#@title Get the author's voice as a source
import random
NAME = str(random.randint(1, 49))
TYPE = "fsd50k" #@param ["", "digit", "dog", "fsd50k"]
CUSTOM_FILEPATH = "" #@param {type: "string"}
if CUSTOM_FILEPATH != "":
    NAME = CUSTOM_FILEPATH
else:
    # it is extremely difficult to find a voice that can download from the internet directly
    if TYPE == "dog":
        !wget -N f"https://huggingface.co/datasets/437aewuh/dog-dataset/resolve/main/dogs/dogs_{NAME:.0000}.wav" -O {NAME}.wav
    elif TYPE == "digit":
        # george, jackson, lucas, nicolas, ...
        !wget -N f"https://github.com/Jakobovski/free-spoken-digit-dataset/raw/master/recordings/0_george_{NAME}.wav" -O {NAME}.wav
    elif TYPE == "fsd50k":
        !wget -N f"https://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/{10000+int(NAME)}.wav" -O {NAME}.wav
    else:
        !wget -N f"https://zunko.jp/sozai/utau/voice_{"kiritan" if NAME < 25 else "itako"}{NAME % 5 + 1}.wav" -O {NAME}.wav
from IPython.display import Audio, display
display(Audio(f"{NAME}.wav"))

In [ ]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svc-fork/audio` directory**
from IPython.display import Audio, display
!svc infer drive/MyDrive/so-vits-svc-fork/audio/{NAME}.wav -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json
display(Audio(f"drive/MyDrive/so-vits-svc-fork/audio/{NAME}.out.wav", autoplay=True))

In [ ]:
##@title Use trained model (with cluster)
!svc infer {NAME}.wav -s speaker -r 0.1 -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json -k drive/MyDrive/so-vits-svc-fork/logs/44k/kmeans.pt
display(Audio(f"{NAME}.out.wav", autoplay=True))

### Pretrained models

In [ ]:
#@title https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/tree/main
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/G_riri_220.pth"
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/config.json"

In [ ]:
!svc infer {NAME}.wav -c config.json -m G_riri_220.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))

In [ ]:
#@title https://huggingface.co/therealvul/so-vits-svc-4.0/tree/main
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/G_166400.pth"
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/config.json"

In [ ]:
!svc infer {NAME}.wav --speaker "Pinkie {neutral}" -c config.json -m G_166400.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))